
# Assignment #2

Pedro Stramantinoli P. Cagliume Gomes 175955

Ruy Castilho Barrichelo 177012


## Importação dos Dados

In [1]:
import pandas as pd
import os

def load_fashionMNIST_train():
    csv_path = os.path.join("Data/fashion-mnist-dataset", "fashion-mnist_train.csv")
    return pd.read_csv(csv_path)

def load_fashionMNIST_tests():
    csv_path = os.path.join("Data/fashion-mnist-dataset", "fashion-mnist_test.csv")
    return pd.read_csv(csv_path)

## Feature Scalling e Normalização

In [2]:
import numpy as np

def normalize(train, test):
    train_mean = np.mean(train, axis=0)
    train = train - train_mean
    test = test - train_mean
    
    return train, test

def scale(X):
    max_array = np.max(X, axis=0)
    X = X / max_array[None, :]
    return X

## Geração de matrizes de Features e Target

### Setup inicial dos dados

In [3]:
import numpy as np

#carrega os dados
data_train = load_fashionMNIST_train()
data_test = load_fashionMNIST_tests()

# Separação em Features e Target

data_train_target = np.array(data_train["label"])
data_test_target = np.array(data_test["label"])

data_train = np.array(data_train)
data_test = np.array(data_test)

data_train = np.delete(data_train, 0, axis=1)
data_test = np.delete(data_test, 0, axis=1)

# Normalização
normalized_data_train, normalized_data_test = normalize(data_train,data_test)

# Scaling
scaled_data_train = scale(normalized_data_train)
scaled_data_test = scale(normalized_data_test)

def getTrainSet():
    return scaled_data_train, data_train_target

def getTestSet():
    return scaled_data_test,data_test_target

## Gerador de conjuntos de Cross Validation

In [4]:
# Cross Validation Generation 

from sklearn.model_selection import train_test_split, KFold

# Retorna um gerador de indices
def generate_sets(TRAINING_DATA,type='kfold'):

    # Cross validation using train_test_split
    if (type == 'split'):
       return train_test_split(TRAINING_DATA,test_size=0.2,random_state=0)

    # Cross validation using K-Fold
    # K = 5, Shuffle = true, Seed = 21
    elif (type == 'kfold'):
        kfold_seed = 21

        kfold = KFold(n_splits=5, shuffle=True, random_state=kfold_seed)
        return kfold.split(TRAINING_DATA)

## Units per Layer

In [5]:
input_layer=784 
units_per_hidden_layer=1024 # 2 to the tenth, since there are 10 classes
output_layer=10

## Learning Rate

In [6]:
learning_rate = 0.01

## Number of Interactions

In [7]:
interactions=1

## Weights and Bias

In [8]:
# Initializes weights randomly

import numpy as np

np.random.seed(21)

def weights1():
    return np.random.rand(input_layer, units_per_hidden_layer)

def weights2():
    return np.random.rand(units_per_hidden_layer, units_per_hidden_layer)
    
def weights3():
    return np.random.rand(units_per_hidden_layer, output_layer)
    
     
# Adds biases, initialized with zeros

def bias1():
    return np.zeros((1, units_per_hidden_layer))

def bias2():
    return np.zeros((1, units_per_hidden_layer))

def bias3():
    return np.zeros((1, output_layer))

def setupNN():
    return weights1(), bias1(), weights2(), bias2(),  weights3(), bias3()

## Activation Functions

### Sigmoid and Loss functions

In [9]:
import numpy as np

def sigmoid(z):
    return 1 / (1+np.exp(-z))

def sigmoid_derivative(a):
    return a*(1 - a)

def loss(h, y):
    return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()


### ReLU

In [10]:
import numpy as np

def ReLU(z):
    z[z<0]=0
    return z

def ReLU_derivative(z):
    z = np.where(z >= 0, 1, 0)
    return z

### Leaky ReLU

In [11]:
import numpy as np

def leaky_ReLU(z):
    z = np.where(z > 0, z, z * 0.01)
    return z

def leaky_ReLU_derivative(z):
    z = np.where(z >= 0, 1, 0.01)
    return z

### Tanh

In [12]:
import numpy as np

def tanh(z):
    return np.tanh(z)

def tanh_derivative(z):
    tan = tanh(z)
    return (1-np.square(tan))

## Softmax

In [13]:
import numpy as np

# Stable softmax
def softmax(z):
    expZ = np.exp(z - np.max(z))
    return expZ / expZ.sum()


## Loss Function - Cross Entropy with Softmax

In [14]:
def cross_entropy_loss(delta, y, length):
    delta[range(length), y] -= 1
    return delta

## Success/Error Percentages

In [15]:
import numpy as np

def error_rate(prediction, y):
    return np.mean( prediction != y )

def success_rate(prediction, y):
    return np.mean( prediction == y )


# Models

#### Different activation functions for hidden layers, but all of them end with the use of Softmax

## Sigmoid

In [16]:
import numpy as np

def sigmoid_model(X, y):

    W1, B1, W2, B2, W3, B3 = setupNN()

    parameters = {}

    for i in range(0, interactions):

        # Forward propagation

        z1 = np.dot(X, W1) + B1

        a1 = sigmoid(z1)

        z2 = np.dot(a1, W2) + B2

        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, W3) + B3
        # Hypothesis
        a3 = softmax(z3)

        # Back propagation
        delta4 = cross_entropy_loss(a3, y, len(X))

        dW3 = np.dot(np.transpose(a2), delta4)
        dB3 = np.sum(delta4, axis=0, keepdims=True)
        dot = np.dot(delta4, np.transpose(W3))
        deriv = sigmoid_derivative(a2)
        delta3 = np.multiply(dot, deriv)

        dW2 = np.dot(np.transpose(a1), delta3)
        dB2 = np.sum(delta3, axis=0, keepdims=True)
        dot = np.dot(delta3, np.transpose(W2))
        deriv = sigmoid_derivative(a1)
        delta2 = np.multiply(dot, deriv)

        dW1 = np.dot(np.transpose(X), delta2)
        dB1 = np.sum(delta2, axis=0, keepdims=True)

        # Updating parameters

        W1 += -learning_rate * dW1
        B1 += -learning_rate * dB1
        W2 += -learning_rate * dW2
        B2 += -learning_rate * dB2
        W3 += -learning_rate * dW3
        B3 += -learning_rate * dB3

        parameters = {'W1': W1, 'B1': B1, 'W2': W2, 'B2': B2, 'W3': W3, 'B3': B3}

    return parameters

def sigmoid_prediction(parameters, X):
    W1, B1, W2, B2, W3, B3 = parameters['W1'], parameters['B1'], parameters['W2'], parameters['B2'], parameters['W3'], parameters['B3']

    z1 = np.dot(X, W1) + B1
    a1 = sigmoid(z1)
    z2 = np.dot(a1, W2) + B2
    a2 = sigmoid(z2)
    z3 = np.dot(a2, W3) + B3
    a3 = softmax(z3)

    return np.argmax(a3, axis=1)

In [17]:
X1, y1 = getTrainSet()
indices_generator = generate_sets(X1)
model_list = []
result_list = []

i=0
for train_index, val_index in indices_generator:
    x1_train = X1[train_index]
    y1_train = y1[train_index]
    x1_val = X1[val_index]
    y1_val = y1[val_index]
     
    parameters = sigmoid_model(x1_train, y1_train)
    model_list.append(parameters)
    
    y1_predict = sigmoid_prediction(parameters, x1_val)
    success = success_rate(y1_predict, y1_val)
    result_list.append(success)
    
    i += 1

for i in range(0, len(result_list)):
    print('Success rate for set', i, ' is: ', result_list[i])

Success rate for set 0  is:  0.10208333333333333
Success rate for set 1  is:  0.09925
Success rate for set 2  is:  0.09483333333333334
Success rate for set 3  is:  0.102
Success rate for set 4  is:  0.10183333333333333


## ReLU

In [20]:
import numpy as np

def relu_model(X, y):

    W1, B1, W2, B2, W3, B3 = setupNN()

    parameters = {}

    for i in range(0, interactions):

        # Forward propagation

        z1 = np.dot(X, W1) + B1

        a1 = ReLU(z1)

        z2 = np.dot(a1, W2) + B2

        a2 = ReLU(z2)
        
        z3 = np.dot(a2, W3) + B3
        
        # Hypothesis
        a3 = softmax(z3)

        # Back propagation
        delta4 = cross_entropy_loss(a3, y, len(X))

        dW3 = np.dot(np.transpose(a2), delta4)
        dB3 = np.sum(delta4, axis=0, keepdims=True)
        dot = np.dot(delta4, np.transpose(W3))
        deriv = ReLU_derivative(a2)
        delta3 = np.multiply(dot, deriv)

        dW2 = np.dot(np.transpose(a1), delta3)
        dB2 = np.sum(delta3, axis=0, keepdims=True)
        dot = np.dot(delta3, np.transpose(W2))
        deriv = ReLU_derivative(z1)
        delta2 = np.multiply(dot, deriv)

        dW1 = np.dot(np.transpose(X), delta2)
        dB1 = np.sum(delta2, axis=0, keepdims=True)

        # Updating parameters

        W1 += -learning_rate * dW1
        B1 += -learning_rate * dB1
        W2 += -learning_rate * dW2
        B2 += -learning_rate * dB2
        W3 += -learning_rate * dW3
        B3 += -learning_rate * dB3

        parameters = {'W1': W1, 'B1': B1, 'W2': W2, 'B2': B2, 'W3': W3, 'B3': B3}

    return parameters

def relu_prediction(parameters, X):
    W1, B1, W2, B2, W3, B3 = parameters['W1'], parameters['B1'], parameters['W2'], parameters['B2'], parameters['W3'], parameters['B3']

    z1 = np.dot(X, W1) + B1
    a1 = ReLU(z1)
    z2 = np.dot(a1, W2) + B2
    a2 = ReLU(z2)
    z3 = np.dot(a2, W3) + B3
    a3 = softmax(z3)

    return np.argmax(a3, axis=1)

In [21]:
X2, y2 = getTrainSet()
indices_generator = generate_sets(X2)
model_list_relu = []
result_list_relu = []

i=0
for train_index, val_index in indices_generator:
    x2_train = X2[train_index]
    y2_train = y2[train_index]
    x2_val = X2[val_index]
    y2_val = y2[val_index]
     
    parameters = relu_model(x2_train, y2_train)
    model_list_relu.append(parameters)
    
    y2_predict = relu_prediction(parameters, x2_val)
    success = success_rate(y2_predict, y2_val)
    result_list_relu.append(success)
    
    i += 1

for i in range(0, len(result_list)):
    print('Success rate for set', i, ' is: ', result_list[i])

Success rate for set 0  is:  0.10208333333333333
Success rate for set 1  is:  0.09925
Success rate for set 2  is:  0.09483333333333334
Success rate for set 3  is:  0.102
Success rate for set 4  is:  0.10183333333333333


## Leaky ReLU

In [22]:
import numpy as np

def leaky_relu_model(X, y):

    W1, B1, W2, B2, W3, B3 = setupNN()

    parameters = {}

    for i in range(0, interactions):

        # Forward propagation

        z1 = np.dot(X, W1) + B1

        a1 = leaky_ReLU(z1)

        z2 = np.dot(a1, W2) + B2

        a2 = leaky_ReLU(z2)
        
        z3 = np.dot(a2, W3) + B3
        
        # Hypothesis
        a3 = softmax(z3)

        # Back propagation
        delta4 = cross_entropy_loss(a3, y, len(X))

        dW3 = np.dot(np.transpose(a2), delta4)
        dB3 = np.sum(delta4, axis=0, keepdims=True)
        dot = np.dot(delta4, np.transpose(W3))
        deriv = leaky_ReLU_derivative(a2)
        delta3 = np.multiply(dot, deriv)

        dW2 = np.dot(np.transpose(a1), delta3)
        dB2 = np.sum(delta3, axis=0, keepdims=True)
        dot = np.dot(delta3, np.transpose(W2))
        deriv = leaky_ReLU_derivative(z1)
        delta2 = np.multiply(dot, deriv)

        dW1 = np.dot(np.transpose(X), delta2)
        dB1 = np.sum(delta2, axis=0, keepdims=True)

        # Updating parameters

        W1 += -learning_rate * dW1
        B1 += -learning_rate * dB1
        W2 += -learning_rate * dW2
        B2 += -learning_rate * dB2
        W3 += -learning_rate * dW3
        B3 += -learning_rate * dB3

        parameters = {'W1': W1, 'B1': B1, 'W2': W2, 'B2': B2, 'W3': W3, 'B3': B3}

    return parameters

def leaky_relu_prediction(parameters, X):
    W1, B1, W2, B2, W3, B3 = parameters['W1'], parameters['B1'], parameters['W2'], parameters['B2'], parameters['W3'], parameters['B3']

    z1 = np.dot(X, W1) + B1
    a1 = leaky_ReLU(z1)
    z2 = np.dot(a1, W2) + B2
    a2 = leaky_ReLU(z2)
    z3 = np.dot(a2, W3) + B3
    a3 = softmax(z3)

    return np.argmax(a3, axis=1)

In [23]:
X3, y3 = getTrainSet()
indices_generator = generate_sets(X3)
model_list_leaky = []
result_list_leaky = []

i=0
for train_index, val_index in indices_generator:
    x3_train = X3[train_index]
    y3_train = y3[train_index]
    x3_val = X3[val_index]
    y3_val = y3[val_index]
     
    parameters = leaky_relu_model(x3_train, y3_train)
    model_list_leaky.append(parameters)
    
    y3_predict = leaky_relu_prediction(parameters, x3_val)
    success = success_rate(y3_predict, y3_val)
    result_list_leaky.append(success)
    
    i += 1

for i in range(0, len(result_list)):
    print('Success rate for set', i, ' is: ', result_list[i])

Success rate for set 0  is:  0.10208333333333333
Success rate for set 1  is:  0.09925
Success rate for set 2  is:  0.09483333333333334
Success rate for set 3  is:  0.102
Success rate for set 4  is:  0.10183333333333333


## Tanh

In [24]:
import numpy as np

def tanh_model(X, y):

    W1, B1, W2, B2, W3, B3 = setupNN()

    parameters = {}

    for i in range(0, interactions):

        # Forward propagation

        z1 = np.dot(X, W1) + B1

        a1 = tanh(z1)

        z2 = np.dot(a1, W2) + B2

        a2 = tanh(z2)
        
        z3 = np.dot(a2, W3) + B3
        
        # Hypothesis
        a3 = softmax(z3)

        # Back propagation
        delta4 = cross_entropy_loss(a3, y, len(X))

        dW3 = np.dot(np.transpose(a2), delta4)
        dB3 = np.sum(delta4, axis=0, keepdims=True)
        dot = np.dot(delta4, np.transpose(W3))
        deriv = tanh_derivative(a2)
        delta3 = np.multiply(dot, deriv)

        dW2 = np.dot(np.transpose(a1), delta3)
        dB2 = np.sum(delta3, axis=0, keepdims=True)
        dot = np.dot(delta3, np.transpose(W2))
        deriv = tanh_derivative(z1)
        delta2 = np.multiply(dot, deriv)

        dW1 = np.dot(np.transpose(X), delta2)
        dB1 = np.sum(delta2, axis=0, keepdims=True)

        # Updating parameters

        W1 += -learning_rate * dW1
        B1 += -learning_rate * dB1
        W2 += -learning_rate * dW2
        B2 += -learning_rate * dB2
        W3 += -learning_rate * dW3
        B3 += -learning_rate * dB3

        parameters = {'W1': W1, 'B1': B1, 'W2': W2, 'B2': B2, 'W3': W3, 'B3': B3}

    return parameters

def tanh_prediction(parameters, X):
    W1, B1, W2, B2, W3, B3 = parameters['W1'], parameters['B1'], parameters['W2'], parameters['B2'], parameters['W3'], parameters['B3']

    z1 = np.dot(X, W1) + B1
    a1 = tanh(z1)
    z2 = np.dot(a1, W2) + B2
    a2 = tanh(z2)
    z3 = np.dot(a2, W3) + B3
    a3 = softmax(z3)

    return np.argmax(a3, axis=1)

In [25]:
X4, y4 = getTrainSet()
indices_generator = generate_sets(X3)
model_list_tanh = []
result_list_tanh = []

i=0
for train_index, val_index in indices_generator:
    x4_train = X4[train_index]
    y4_train = y4[train_index]
    x4_val = X4[val_index]
    y4_val = y4[val_index]
     
    parameters = leaky_relu_model(x4_train, y4_train)
    model_list_tanh.append(parameters)
    
    y4_predict = leaky_relu_prediction(parameters, x4_val)
    success = success_rate(y4_predict, y4_val)
    result_list_tanh.append(success)
    
    i += 1

for i in range(0, len(result_list)):
    print('Success rate for set', i, ' is: ', result_list[i])

Success rate for set 0  is:  0.10208333333333333
Success rate for set 1  is:  0.09925
Success rate for set 2  is:  0.09483333333333334
Success rate for set 3  is:  0.102
Success rate for set 4  is:  0.10183333333333333


# Test

In [26]:
# Xt, yt = getTestSet()

# yt_predict = insertmodelhere(insertparameters here, Xt)
# success = success_rate(yt_predict, yt)

# print('Success rate for the test set is: ', success)